# Inside Unsupervised Learning: Generative Models and Recommender Systems
## Explore generative models and build movie recommender systems
### by Ankur A. Patel + O'Reilly Media, Inc.

## Overview
In this notebook, we will explore:
#1 The difference between discriminative and generative models
#2 Why generative models are so powerful
#3 The various types of generative models used today
#4 How to train a simple generative model (restricted Boltzmann machines)

Specifically, we will train RBMs to build a movie recommender system. After this course, you should be familiar with generative models and know a bit about how to train recommender systems in your field.

## Data Preparation
Load in the movie ratings dataset and prepare it.

We will use the MovieLens 20M Dataset. The original ratings file has 20,000,263 ratings, 27,278 movies, and 138,493 users.

We will use a reduced version of this data for this course. Our reduced version has data for the top 1,000 movies and a sample of 1,000 users.

Since not all 1,000 users rated all 1,000 movies, there are 664,550 ratings in total.

For each movie review, we have the original movieID, the original userID, the movie rating the user assigned, and the timestamp. We have two additional fields, the new movieID and the new userID, which are just reindexed versions of the original movieID and the original userID for the reduced dataset.

In [ ]:
'''Main'''
import numpy as np
import pandas as pd
import os, time, re
import pickle, gzip, datetime
from datetime import datetime

'''Data Viz'''
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl

%matplotlib inline

'''Data Prep and Model Evaluation'''
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, mean_squared_error

'''Algos'''
import lightgbm as lgb

'''TensorFlow and Keras'''
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout
from keras.layers import BatchNormalization, Input, Lambda
from keras.layers import Embedding, Flatten, dot
from keras import regularizers
from keras.losses import mse, binary_crossentropy

In [ ]:
# Load the data
os.chdir('/home/jovyan/')
current_path = os.getcwd()
pickle_file = '/data/movielens_data/ratingReducedPickle'
ratingDF = pd.read_pickle(current_path + pickle_file)

In [ ]:
# View 5 rows of the data
ratingDF.head()

In [ ]:
# Calculate summary statistics on reduced dataset
n_users = ratingDF.userId.unique().shape[0]
n_movies = ratingDF.movieId.unique().shape[0]
n_ratings = len(ratingDF)
avg_ratings_per_user = n_ratings/n_users

print('Number of unique users: ', n_users)
print('Number of unique movies: ', n_movies)
print('Number of total ratings: ', n_ratings)
print('Average number of ratings per user: ', avg_ratings_per_user)

In [ ]:
# Split into validation and test, such that each is 5% of the dataset
X_train, X_test = train_test_split(ratingDF, test_size=0.10, \
                                   shuffle=True, random_state=2018)

X_validation, X_test = train_test_split(X_test, test_size=0.50, \
                                        shuffle=True, random_state=2018)

In [ ]:
# Confirm size of train, validation, and test datasets
print('Size of train set: ', len(X_train))
print('Size of validation set: ', len(X_validation))
print('Size of test set: ', len(X_test))

In [ ]:
# Generate ratings matrix for train
ratings_train = np.zeros((n_users, n_movies))
for row in X_train.itertuples():
    ratings_train[row[6]-1, row[5]-1] = row[3]

In [ ]:
# Calculate sparsity of the train ratings matrix
sparsity = float(len(ratings_train.nonzero()[0]))
sparsity /= (ratings_train.shape[0] * ratings_train.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))

In [ ]:
# Generate ratings matrix for validation
ratings_validation = np.zeros((n_users, n_movies))
for row in X_validation.itertuples():
    ratings_validation[row[6]-1, row[5]-1] = row[3]

In [ ]:
# Generate ratings matrix for test
ratings_test = np.zeros((n_users, n_movies))
for row in X_test.itertuples():
    ratings_test[row[6]-1, row[5]-1] = row[3]

In [ ]:
# Calculate sparsity of the validation ratings matrix
sparsity = float(len(ratings_validation.nonzero()[0]))
sparsity /= (ratings_validation.shape[0] * ratings_validation.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))

## Experiment #1 - Naive Recommender System A
Assign naive 3.5 rating to each missing movie review and calculate the mean squared error.

In [ ]:
# Store actual ratings for the validation set
actual_validation = ratings_validation[ratings_validation.nonzero()].flatten()

# Assign "naive" 3.5 rating to missing values in the validation set
pred_validation = np.zeros((len(X_validation),1))
pred_validation[pred_validation==0] = 3.5

# Calculate mean square error using this naive prediction
naive_prediction = mean_squared_error(pred_validation, actual_validation)
print('Mean squared error using naive prediction:', naive_prediction)

## Experiment #2 - Naive Recommender System B
Predict a user's rating based on user's average rating for all other movies the user has rated.

In [ ]:
# Average user rating recommender system
ratings_validation_prediction = np.zeros((n_users, n_movies))
i = 0
for row in ratings_train:
    ratings_validation_prediction[i][ratings_validation_prediction[i]==0] \
        = np.mean(row[row>0])
    i += 1

pred_validation = ratings_validation_prediction \
    [ratings_validation.nonzero()].flatten()
user_average = mean_squared_error(pred_validation, actual_validation)
print('Mean squared error using user average:', user_average)

## Experiment #3 - Naive Recommender System C
Predict a user's rating for a movie based on the average rating other users have given that movie.

In [ ]:
# Average movie rating recommender system
ratings_validation_prediction = np.zeros((n_users, n_movies)).T
i = 0
for row in ratings_train.T:
    ratings_validation_prediction[i][ratings_validation_prediction[i]==0] \
        = np.mean(row[row>0])
    i += 1

ratings_validation_prediction = ratings_validation_prediction.T
pred_validation = ratings_validation_prediction \
    [ratings_validation.nonzero()].flatten()
movie_average = mean_squared_error(pred_validation.astype(np.float64), \
                                   actual_validation.astype(np.float64))
print('Mean squared error using movie average:', movie_average)

## Experiment #4 - Recommender System using Matrix Factorization
We will use 1 latent factor, but you could experiment with a different number of latent factors.

In [ ]:
n_latent_factors = 1

user_input = Input(shape=[1], name='user')
user_embedding = Embedding(input_dim=n_users + 1, \
                           output_dim=n_latent_factors, \
                           name='user_embedding')(user_input)
user_vec = Flatten(name='flatten_users')(user_embedding)

movie_input = Input(shape=[1], name='movie')
movie_embedding = Embedding(input_dim=n_movies + 1, \
                            output_dim=n_latent_factors,
                            name='movie_embedding')(movie_input)
movie_vec = Flatten(name='flatten_movies')(movie_embedding)

product = dot([movie_vec, user_vec], axes=1)
model = Model(inputs=[user_input, movie_input], outputs=product)
model.compile('adam', 'mean_squared_error')

In [ ]:
history = model.fit(x=[X_train.newUserId, X_train.newMovieId], \
                    y=X_train.rating, epochs=10, \
                    validation_data=([X_validation.newUserId, \
                    X_validation.newMovieId], X_validation.rating), \
                    verbose=1)

In [ ]:
pd.Series(history.history['val_loss'][:]).plot(logy=False)
plt.xlabel("Epoch")
plt.ylabel("Validation Error")
print('Minimum MSE: ', min(history.history['val_loss']))

preds = model.predict(x=[X_validation.newUserId, X_validation.newMovieId])

In [ ]:
# MSE on train
preds = model.predict(x=[X_train.newUserId, X_train.newMovieId])
mean_squared_error(preds, X_train.rating)

In [ ]:
# MSE on validation
preds = model.predict(x=[X_validation.newUserId, X_validation.newMovieId])
mean_squared_error(preds, X_validation.rating)

In [ ]:
# MSE on test
preds = model.predict(x=[X_test.newUserId, X_test.newMovieId])
mean_squared_error(preds, X_test.rating)

## Experiment #5 - Recommender System using RBMs

In [ ]:
# Define RBM class
class RBM(object):
    
    def __init__(self, input_size, output_size, 
                 learning_rate, epochs, batchsize):
        # Define hyperparameters
        self._input_size = input_size
        self._output_size = output_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchsize = batchsize
        
        # Initialize weights and biases using zero matrices
        self.w = np.zeros([input_size, output_size], dtype=np.float32)
        self.hb = np.zeros([output_size], dtype=np.float32)
        self.vb = np.zeros([input_size], dtype=np.float32)

    def prob_h_given_v(self, visible, w, hb):
        return tf.nn.sigmoid(tf.matmul(visible, w) + hb)

    def prob_v_given_h(self, hidden, w, vb):
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w)) + vb)
    
    def sample_prob(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random_uniform(tf.shape(probs))))

    def train(self, X):
        _w = tf.placeholder(tf.float32, [self._input_size, self._output_size])
        _hb = tf.placeholder(tf.float32, [self._output_size])
        _vb = tf.placeholder(tf.float32, [self._input_size])
        
        prv_w = np.zeros([self._input_size, self._output_size], dtype=np.float32)
        prv_hb = np.zeros([self._output_size], dtype=np.float32)
        prv_vb = np.zeros([self._input_size], dtype=np.float32)
        
        cur_w = np.zeros([self._input_size, self._output_size], dtype=np.float32)
        cur_hb = np.zeros([self._output_size], dtype=np.float32)
        cur_vb = np.zeros([self._input_size], dtype=np.float32)
        
        v0 = tf.placeholder(tf.float32, [None, self._input_size])
        h0 = self.sample_prob(self.prob_h_given_v(v0, _w, _hb))
        v1 = self.sample_prob(self.prob_v_given_h(h0, _w, _vb))
        h1 = self.prob_h_given_v(v1, _w, _hb)
        
        positive_grad = tf.matmul(tf.transpose(v0), h0)
        negative_grad = tf.matmul(tf.transpose(v1), h1)
        
        update_w = _w + self.learning_rate * \
            (positive_grad - negative_grad) / tf.to_float(tf.shape(v0)[0])
        update_vb = _vb + self.learning_rate * tf.reduce_mean(v0 - v1, 0)
        update_hb = _hb + self.learning_rate * tf.reduce_mean(h0 - h1, 0)
        
        err = tf.reduce_mean(tf.square(v0 - v1))
        
        error_list = []
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            
            for epoch in range(self.epochs):
                for start, end in zip(range(0, len(X), \
                        self.batchsize),range(self.batchsize,len(X), \
                                              self.batchsize)):
                    batch = X[start:end]
                    cur_w = sess.run(update_w, feed_dict={v0: batch, \
                                    _w: prv_w, _hb: prv_hb, _vb: prv_vb})
                    cur_hb = sess.run(update_hb, feed_dict={v0: batch, \
                                    _w: prv_w, _hb: prv_hb, _vb: prv_vb})
                    cur_vb = sess.run(update_vb, feed_dict={v0: batch, \
                                    _w: prv_w, _hb: prv_hb, _vb: prv_vb})
                    prv_w = cur_w
                    prv_hb = cur_hb
                    prv_vb = cur_vb
                error = sess.run(err, feed_dict={v0: X, \
                                _w: cur_w, _vb: cur_vb, _hb: cur_hb})
                print ('Epoch: %d' % epoch,'reconstruction error: %f' % error)
                error_list.append(error)
            self.w = prv_w
            self.hb = prv_hb
            self.vb = prv_vb
            return error_list

    def rbm_output(self, X):
        
        input_X = tf.constant(X)
        _w = tf.constant(self.w)
        _hb = tf.constant(self.hb)
        _vb = tf.constant(self.vb)
        out = tf.nn.sigmoid(tf.matmul(input_X, _w) + _hb)
        visibleGen = tf.nn.sigmoid(tf.matmul(out, tf.transpose(_w)) + _vb)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(out)
            sess.run(visibleGen)
            return sess.run(out), sess.run(visibleGen)

In [ ]:
# Begin the training cycle

# Convert inputX into float32
inputX = ratings_train/5.0
inputX = inputX.astype(np.float32)

# Define the parameters of the RBMs we will train
rbm=RBM(1000,10,0.1,300,200)

In [ ]:
# Train RBM model
err = rbm.train(inputX)
outputX, reconstructedX = rbm.rbm_output(inputX)

In [ ]:
# Plot reconstruction errors
pd.Series(err).plot(logy=False)
plt.xlabel("Epoch")
plt.ylabel("Reconstruction Error")

In [ ]:
# Find scaling factor based on training set
actual_train = ratings_train[ratings_train.nonzero()].flatten()
train_mean = np.mean(actual_train)

In [ ]:
# Calculate MSE on train set
predictionsArray = reconstructedX
pred_train = \
    predictionsArray[ratings_train.nonzero()].flatten()
pred_train = pred_train * train_mean/np.mean(pred_train)
actual_train = \
    ratings_train[ratings_train.nonzero()].flatten()

rbm_prediction = mean_squared_error(pred_train, actual_train)
print('Mean squared error using RBM prediction:', rbm_prediction)

In [ ]:
# Calculate MSE on validation set
predictionsArray = reconstructedX
pred_validation = \
    predictionsArray[ratings_validation.nonzero()].flatten()
pred_validation = pred_validation * train_mean/np.mean(pred_validation)
actual_validation = \
    ratings_validation[ratings_validation.nonzero()].flatten()

rbm_prediction = mean_squared_error(pred_validation, actual_validation)
print('Mean squared error using RBM prediction:', rbm_prediction)

In [ ]:
# View generated predictions on validation set
pred_validation

In [ ]:
# View actual ratings in validation set
actual_validation

## Exercise
Train the RBM with an input layer of 1,000 nodes, a output layer of 50 nodes, a learning rate of 0.3, and a batch size of 32 for 100 epochs.

Then, evaluate the performance on the train set and validation set.

In [ ]:
# Begin the training cycle

# Convert inputX into float32
inputX = ratings_train/5.0
inputX = inputX.astype(np.float32)

# Define the parameters of the RBMs we will train
rbm=RBM(#input nodes, #output nodes, #learning rate, #epochs, #batch size)

In [ ]:
# Train RBM model
err = rbm.train(#Fill in)
outputX, reconstructedX = rbm.rbm_output(#Fill in)

In [ ]:
# Plot reconstruction errors
pd.Series(err).plot(logy=False)
plt.xlabel("Epoch")
plt.ylabel("Reconstruction Error")

In [ ]:
# Calculate MSE on train set
predictionsArray = reconstructedX
pred_train = \
    predictionsArray[ratings_train.nonzero()].flatten()
pred_train = pred_train * train_mean/np.mean(pred_train)
actual_train = \
    ratings_train[ratings_train.nonzero()].flatten()

rbm_prediction = mean_squared_error(pred_train, actual_train)
print('Mean squared error using RBM prediction:', rbm_prediction)

In [ ]:
# Calculate MSE on validation set
predictionsArray = reconstructedX
pred_validation = \
    predictionsArray[ratings_validation.nonzero()].flatten()
pred_validation = pred_validation * train_mean/np.mean(pred_validation)
actual_validation = \
    ratings_validation[ratings_validation.nonzero()].flatten()

rbm_prediction = mean_squared_error(pred_validation, actual_validation)
print('Mean squared error using RBM prediction:', rbm_prediction)

## Answers to Exercise

In [ ]:
# Begin the training cycle

# Convert inputX into float32
inputX = ratings_train/5.0
inputX = inputX.astype(np.float32)

# Define the parameters of the RBMs we will train
rbm=RBM(1000, 50, 0.3, 100, 32)

In [ ]:
# Train RBM model
err = rbm.train(inputX)
outputX, reconstructedX = rbm.rbm_output(inputX)

In [ ]:
# Plot reconstruction errors
pd.Series(err).plot(logy=False)
plt.xlabel("Epoch")
plt.ylabel("Reconstruction Error")

In [ ]:
# Calculate MSE on train set
predictionsArray = reconstructedX
pred_train = \
    predictionsArray[ratings_train.nonzero()].flatten()
pred_train = pred_train * train_mean/np.mean(pred_train)
actual_train = \
    ratings_train[ratings_train.nonzero()].flatten()

rbm_prediction = mean_squared_error(pred_train, actual_train)
print('Mean squared error using RBM prediction:', rbm_prediction)

In [ ]:
# Calculate MSE on validation set
predictionsArray = reconstructedX
pred_validation = \
    predictionsArray[ratings_validation.nonzero()].flatten()
pred_validation = pred_validation * train_mean/np.mean(pred_validation)
actual_validation = \
    ratings_validation[ratings_validation.nonzero()].flatten()

rbm_prediction = mean_squared_error(pred_validation, actual_validation)
print('Mean squared error using RBM prediction:', rbm_prediction)

## Conclusion
In this notebook, we explored generative models and RBMs.

We then built non-generative movie recommender systems.

And then we built a generative model-based movie recommender system using RBMs.

The generative model-based movie recommender system did not fare as well as the non-generative movie recommender systems.

But, there is a lot of room for improvement. This is what makes building great generative models and recommender systems difficult.

Congratulations, you've finished this course! 
Go build more recommender systems in your field!

The next course in the Inside Unsupervised Learning series is Anomaly Detection Using Unsupervised Learning.
https://learning.oreilly.com/live-training/courses/inside-unsupervised-learning-anomaly-detection-using-dimensionality-reduction/0636920289654/

You could also learn more about Unsupervised Learning in my book, Hands-on Unsupervised Learning Using Python.
https://www.unsupervisedlearningbook.com/

Finally, there is a Slack commmunity for you to join.
https://join.slack.com/t/unsupervisedbook/shared_invite/enQtNjQzMjMyMjAyMjQyLTE3MWZlNTU0N2U3Zjg2MzNjMjM5MTI4YmIxY2ExOTIzNzA0ZTVmYjAyYWJhMjE1ZTc3NDRiNDIwOGM5YmY0M2E